In [1]:
import sys
import numpy as np
import cv2

In [3]:
lenna = cv2.imread('../data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if lenna is None:
    print('Image load failed!')
    sys.exit()

In [4]:
building = cv2.imread('../data/building.jpg', cv2.IMREAD_GRAYSCALE)

if building is None:
    print('Image load failed!')
    sys.exit()

In [3]:
dial = cv2.imread('../data/dial.jpg')

if dial is None:
    print('Image load failed!')
    sys.exit()

## Sobel filter

In [5]:
dx = cv2.Sobel(lenna, -1, 1,0, delta=128)
dy = cv2.Sobel(lenna, -1, 0,1, delta=128)

In [6]:
cv2.imshow('src', lenna)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()

cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/j-hyungjun/venv/OpenCV_venv/lib/python3.10/site-packages/cv2/qt/plugins"


## Sobel filter Edge extract

In [7]:
dx = cv2.Sobel(lenna, cv2.CV_32F, 1,0)
dy = cv2.Sobel(lenna, cv2.CV_32F, 0,1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0,255).astype(np.uint8)

dst = np.zeros(lenna.shape[:2], np.uint8)
_, dst = cv2.threshold(mag, 120,255, cv2.THRESH_BINARY)

In [8]:
cv2.imshow('src', lenna)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

## Canny edge extract

In [9]:
dst = cv2.Canny(building, 50,150)

cv2.imshow('src', building)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

## Hough transform line extract

In [10]:
edges = cv2.Canny(building, 50,150)
lines = cv2.HoughLinesP(edges, 1, np.pi/180., 160,
                        minLineLength=50, maxLineGap=5)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0,0,255), 2, cv2.LINE_AA)

In [11]:
cv2.imshow('src', building)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

## Hough transform circle extract

In [7]:
gray = cv2.cvtColor(dial, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0,0), 1.0)

In [10]:
def on_trackbar(pos):
    rmin = cv2.getTrackbarPos('minRadius', 'img')
    rmax = cv2.getTrackbarPos('maxRadius', 'img')
    th = cv2.getTrackbarPos('threshold', 'img')

    if th > 0:
        circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1,50,
                               param1=120, param2=th, minRadius=rmin, maxRadius=rmax)
        dst = dial.copy()
        if circles is not None:
            for i in range(circles.shape[1]):
                cx, cy, radius = circles[0][i]
                cv2.circle(dst, (int(cx), int(cy)), int(radius), (0,0,255), 2, cv2.LINE_AA)
                cv2.imshow('img', dst)

In [11]:
cv2.imshow('img', dial)
cv2.createTrackbar('minRadius', 'img', 0,100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0,150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 1,100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)
cv2.waitKey()

cv2.destroyAllWindows()